In [1]:
import dwave_sapi2
from dwave_sapi2.local import local_connection
from dwave_sapi2.remote import RemoteConnection
from dwave_sapi2.core import async_solve_ising, await_completion, solve_ising
from dwave_sapi2.util import get_hardware_adjacency, qubo_to_ising
from dwave_sapi2.embedding import find_embedding, embed_problem, unembed_answer
import time
import dwave_sapi2.remote
import sys
import matplotlib.pyplot as plt

import numpy as np
import scipy.io as sio
import math 
import random
from numpy import linalg as LA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.neural_network import MLPClassifier
import pandas as pd

In [2]:
conn = dwave_sapi2.remote.RemoteConnection('https://cloud.dwavesys.com/sapi','lanl-871046202c27322e8450da36b7b35653c94c1115')
print conn.solver_names()

['DW_2000Q_2_1', 'DW_2000Q_5', 'DW_2000Q_VFYC_2_1', 'c4-sw_optimize', 'DW_2000Q_VFYC_5', 'c4-sw_sample']


In [3]:
cd Desktop/LANL/AFRL-DISC Data/Blackbox brief for LANL

C:\Users\khenk\Desktop\LANL\AFRL-DISC Data\Blackbox brief for LANL


In [4]:
sPCA= pd.read_csv('sPCA_32.csv', sep=',', encoding = 'utf8')
data = sPCA.values

In [5]:
mat = sio.loadmat('Fashion_Downsampled_4Bit.mat')
Down_Sampled_4bit = mat['Down_Sampled_4bit']
data = Down_Sampled_4bit.reshape(len(Down_Sampled_4bit),len(Down_Sampled_4bit[0])**2)

In [6]:
data = data.astype(np.float32)/255

In [7]:
#normalize data
for i in range(len(data)):

    data[i]=data[i]/LA.norm(data[i])

In [8]:
def sparse_loss(I,Phi,sparse_representation_coefficients,lam):
    Energy = .5*LA.norm(I-np.dot(Phi,sparse_representation_coefficients))**2 +lam*np.count_nonzero(sparse_representation_coefficients)
    return Energy

def reconstruction_loss(I,Phi,sparse_representation_coefficients):
    Loss = LA.norm(I-np.dot(Phi,sparse_representation_coefficients))**2
    return Loss

In [9]:
#Creating Random Phi Matrix
Neurons = 32
RandomTrainingSample_Index = random.sample(range(len(data)),Neurons)
Rand_Features = data[RandomTrainingSample_Index,:]


In [10]:
#Normalize Phi
Phi = Rand_Features.T
Phi = np.asmatrix(Phi)

for i in range(Neurons):
    print((math.sqrt(Phi[:,i].T*Phi[:,i])))
    Phi[:,i]= Phi[:,i]/(math.sqrt(Phi[:,i].T*Phi[:,i]))
    print((math.sqrt(Phi[:,i].T*Phi[:,i])))

1.0000000596
1.0000000596
0.999999940395
1.0
1.0
1.0
0.999999970198
1.0000000596
0.999999940395
1.0000000596
1.0
1.0
0.999999880791
1.0
0.999999850988
1.00000011921
1.0
1.0
0.999999970198
1.0000000596
1.0
1.0
0.999999970198
1.0
1.0
1.0
1.0000000596
1.0000000596
1.0
1.0
0.999999970198
1.0000000596
1.0
1.0
1.0
1.0
0.999999940395
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0000000596
1.0000000596
0.999999940395
1.0
1.0000000596
1.0000000596
1.0
1.0
1.0
1.0
1.0
1.0
1.0000000596
1.0000000596
0.999999940395
1.00000011921
1.0
1.0


In [11]:
sio.savemat('Phi_Random_Lambda_.45_RSGD_sPCA.mat', {'Phi':Phi})

In [12]:
#Pull Random images for SGD Training
Num_Ran_Sam =10000 
RandomTrainingSample_Index = random.sample(range(len(data)),Num_Ran_Sam)
Data_Patches = data[RandomTrainingSample_Index,:]

In [13]:
Q = .5*np.dot(np.squeeze(Phi.T),np.squeeze(Phi))
lam = .45
lam_vector = np.full((len(Phi.T), 1), lam, dtype=np.float)
h_vector = np.add(-np.dot(Phi.T,Data_Patches[0,:].T),np.squeeze(np.add(lam_vector,np.full((len(Phi.T), 1), lam, dtype=np.float))))
h_vector = np.squeeze(np.asarray(h_vector))
#Replace Diagonal of Q with corresponding h values    
for i in range(0,len(Q)):
    Q[i,i]=h_vector[i]

#Create Dictionary of Upper Triangle part of Q with Diagonal Elements
QDictionary ={}
for i in range(0,len(Q)):
    for j in range(i,len(Q)):
            QDictionary[(i,j)] = Q[i,j]
            #print(i,j,Q[i,j])

In [14]:
#Finding lowest Energy Embedding    
Embedding_Trials = 5
Unembedded_Solutions=[]
Embeddings =[]
EmbeddingTime=[]
J0 =[]
JC=[]
Coefficients_sol =[]
Coefficients_embsol=[]
solver = conn.get_solver('DW_2000Q_2_1') 
A = get_hardware_adjacency(solver)


for i in range(Embedding_Trials):
    
        (h_, J_, ising_offset) = qubo_to_ising(QDictionary)
        print(np.asarray(h_).shape)
        #[h_,J_, offsetarg]= qubo_to_ising[Q]

        #solver = local_connection.get_solver("c4-sw_optimize") # dont select


        # find and print embeddings for problem graph
        t = time.time()

        embeddings = find_embedding(J_, A, verbose=1)
        elapsed = time.time() - t
        EmbeddingTime.append(elapsed)
        print "elapsed time:", elapsed
        #print "embeddings are: ", embeddings
        Embeddings.append(embeddings)

        # embed the problem into solver graph
        (h0, j0, jc, new_emb) = embed_problem(h_, J_, embeddings, A) #update to have multiple solutions and save embeddings
#         print "embedded problem result:\nj0: ", j0
#         print "jc: ", jc
        J0.append(j0)
        JC.append(jc)

        # find unembedded results for chain strengths -0.5, -1.0, -2.0

        Solutions =[]
        Answers=[]
        for chain_strength in (-0.5, -1.0, -2.0):
                # set chain strength values
            jc = dict.fromkeys(jc, chain_strength)

            # create new J array concatenating j0 with jc
            emb_j = j0.copy()
            emb_j.update(jc)

                # solve embedded problem

            answer = solve_ising(solver, h0, emb_j, num_reads=10)# save results in an array
            Answers.append(answer['energies'])

                # unembed and print result of the form:
                # solution [solution #]
                # var [var #] : [var value] ([qubit index] : [original qubit value] ...)
            result = unembed_answer(answer['solutions'], new_emb, broken_chains="minimize_energy", h=h_, j=J_)
            Solutions.append(result)
            #Solutions.append(result['energies']) DOESNT HAVE energies AS AN OUTPUT

#             print(result)
#             print "result for chain strength = ", chain_strength
#             for i, (embsol, sol) in enumerate(zip(answer['solutions'], result)):
#                 print "solution", i
#                 #save solution coefficients
#                 for j, emb in enumerate(embeddings):
#                     Coefficients_sol.append(sol[j])
#                     print "var %d: %d (" % (j, sol[j]),
#                     for k in emb:
#                         Coefficients_embsol.append(embsol[k])
#                         print "%d:%d" % (k, embsol[k]),
#                     print ")"
            #print "Solutions", Solutions
        New_Solutions=[]
        #Convert to {0,1}
        for j in range(0,3):
            x = np.asarray(Solutions[j][0])
            for i in range(0,len(x)):
                if x[i] == -1:
                     x[i] = 0
            New_Solutions.append(x)
        Unembedded_Solutions.append(New_Solutions)
        if __name__ == "__main__":
            if len(sys.argv) == 1:
                embedding_example()
            else:
                print "Usage: "
                #print "%s: Find embedding for k_6 structured graph, embed problem, solve problem, unembed answer" % sys.argv[0]

(32L,)
component 0, try 0:
max overfill = 2, num max overfills = 10
max overfill = 1, num max overfills = 453
max overfill = 1, num max overfills = 453
Embedding found. Minimizing chains...
max chain size = 19, num max chains = 2, qubits used = 453
max chain size = 19, num max chains = 1, qubits used = 462
max chain size = 19, num max chains = 1, qubits used = 462
max chain size = 17, num max chains = 4, qubits used = 411
max chain size = 17, num max chains = 3, qubits used = 402
max chain size = 17, num max chains = 3, qubits used = 402
elapsed time: 9.62599992752
Usage: 
(32L,)
component 0, try 0:
max overfill = 2, num max overfills = 8
max overfill = 1, num max overfills = 418
max overfill = 1, num max overfills = 418
max overfill = 1, num max overfills = 401
max overfill = 1, num max overfills = 401
Embedding found. Minimizing chains...
max chain size = 16, num max chains = 2, qubits used = 401
max chain size = 16, num max chains = 1, qubits used = 399
max chain size = 16, num max 

In [15]:
All_Energies = []
for i in range(len(Unembedded_Solutions)):
    Energy_Chain = []
    for j in range(3):
        Energy_Chain.append(sparse_loss(Data_Patches[0],Phi,np.asmatrix(Unembedded_Solutions[i][j]).T,lam))
    index_min = np.argmin(np.asarray(Energy_Chain))
    All_Energies.append(Energy_Chain[index_min])
index_min = np.argmin(np.asarray(All_Energies))
embeddings = Embeddings[index_min]

In [ ]:
#SGD Loop to update Dictionary

mini_batch_size = 1000
tol = 10**(-32)
max_iterations = 10000
eta =.005 #Learning Rate
lam = .35
lam_vector = np.full((len(Phi.T), 1), lam, dtype=np.float)
count = 0
Previous_Gradient = 0
momentum_term = .2

#Energy_Min = 20


#Total_Energies = []

solver = conn.get_solver('DW_2000Q_2_1') 
A = get_hardware_adjacency(solver)
#Non_Zero_Coefficients =[]

for iteration in range(max_iterations):
    
        
    #Random Draw from Data
    Indexing = random.sample(range(len(Data_Patches)), mini_batch_size)
    
    Neuron_Activation =[]
    Gradients =[]
    Unembedded_Solutions_Training= []
    t1 = time.time()
    New_Solutions = []
    Temp_Data_Patches=[]
    Q = .5*np.dot(np.squeeze(Phi.T),np.squeeze(Phi))
    

    J0 =[]
    JC=[]
    Coefficients_sol =[]
    Coefficients_embsol=[]
    image_number = 0
    for sample in Indexing:
        
        h_vector = np.add(-np.dot(Phi.T,Data_Patches[sample,:].T),np.squeeze(np.add(lam_vector,np.full((len(Phi.T), 1), lam, dtype=np.float))))
        h_vector = np.squeeze(np.asarray(h_vector))
        #Replace Diagonal of Q with corresponding h values    
        for i in range(0,len(Q)):
            Q[i,i]=h_vector[i]

        #Create Dictionary of Upper Triangle part of Q with Diagonal Elements
        QDictionary ={}
        for i in range(0,len(Q)):
            for j in range(i,len(Q)):
                    QDictionary[(i,j)] = Q[i,j]
                    #print(i,j,Q[i,j])

        (h_, J_, ising_offset) = qubo_to_ising(QDictionary)
            #[h_,J_, offsetarg]= qubo_to_ising[Q]

#         embeddings = find_embedding(J_, A, verbose=1)
        (h0, j0, jc, new_emb) = embed_problem(h_, J_, embeddings, A)

        Solutions =[]
        Answers=[]
        
        Temp_Data_Patches.append(Data_Patches[sample,:].T)

            # solve embedded problem
        
        Solutions =[]
        Answers=[]
        for chain_strength in (-0.5, -1.0, -2.0):
                # set chain strength values
            jc = dict.fromkeys(jc, chain_strength)

            # create new J array concatenating j0 with jc
            emb_j = j0.copy()
            emb_j.update(jc)

                # solve embedded problem
            try:
                answer = solve_ising(solver, h0, emb_j, num_reads=10)# save results in an array
                Answers.append(answer['energies'])
            except RuntimeError:
                time.sleep(30)
                print('Runtime Error: Resubmitting Problem to D-Wave')
                try:
                    answer = solve_ising(solver, h0, emb_j, num_reads=10)# save results in an array
                    Answers.append(answer['energies'])

                except RuntimeError:
                    time.sleep(30)
                    print('Runtime Error: Resubmitting Problem to D-Wave')
                    try:
                        answer = solve_ising(solver, h0, emb_j, num_reads=10)# save results in an array
                        Answers.append(answer['energies'])
                    except RuntimeError:
                        time.sleep(30)
                        print('Runtime Error: Resubmitting Problem to D-Wave')
                        try:
                            answer = solve_ising(solver, h0, emb_j, num_reads=10)# save results in an array
                            Answers.append(answer['energies'])

                        except RuntimeError:
                            time.sleep(30)
                            print('Runtime Error: Resubmitting Problem to D-Wave')
                            try:
                                answer = solve_ising(solver, h0, emb_j, num_reads=10)# save results in an array
                                Answers.append(answer['energies'])

                            except RuntimeError:
                                time.sleep(30)
                                print('Runtime Error: Resubmitting Problem to D-Wave')
                                try:
                                    answer = solve_ising(solver, h0, emb_j, num_reads=10)# save results in an array
                                    Answers.append(answer['energies'])

                                except RuntimeError:
                                    time.sleep(30)
                                    print('Runtime Error: Resubmitting Problem to D-Wave')
                                    try:
                                        answer = solve_ising(solver, h0, emb_j, num_reads=10)# save results in an array
                                        Answers.append(answer['energies'])

                                    except RuntimeError:
                                        time.sleep(30)
                                        print('Runtime Error: Resubmitting Problem to D-Wave')
                                        try:
                                            answer = solve_ising(solver, h0, emb_j, num_reads=10)# save results in an array
                                            Answers.append(answer['energies'])

                                        except RuntimeError:
                                            time.sleep(30)
                                            print('Runtime Error: Resubmitting Problem to D-Wave')
                                            try:
                                                answer = solve_ising(solver, h0, emb_j, num_reads=10)# save results in an array
                                                Answers.append(answer['energies'])

                                            except RuntimeError:
                                                time.sleep(30)
                                                print('Runtime Error: Resubmitting Problem to D-Wave')
                                                answer = solve_ising(solver, h0, emb_j, num_reads=10)# save results in an array
                                                Answers.append(answer['energies'])



                # unembed and print result of the form:
                # solution [solution #]
                # var [var #] : [var value] ([qubit index] : [original qubit value] ...)
            result = unembed_answer(answer['solutions'], new_emb, broken_chains="minimize_energy", h=h_, j=J_)
            Solutions.append(result)
            #Solutions.append(result['energies']) DOESNT HAVE energies AS AN OUTPUT

            #print(result)
            #print "result for chain strength = ", chain_strength
            for i, (embsol, sol) in enumerate(zip(answer['solutions'], result)):
                #print "solution", i
                #save solution coefficients
                for j, emb in enumerate(embeddings):
                    Coefficients_sol.append(sol[j])
                    #print "var %d: %d (" % (j, sol[j]),
                    for k in emb:
                        Coefficients_embsol.append(embsol[k])
                        #print "%d:%d" % (k, embsol[k]),
                    #print ")"
            #print "Solutions", Solutions
        New_Solutions=[]
        #Convert to {0,1}
        for j in range(0,3):
            x = np.asarray(Solutions[j][0])
            x = (x+1)/2
    #             for i in range(0,Phi.shape[1]):
    #                 if x[i] == -1:
    #                      x[i] = 0
            New_Solutions.append(x)
        New_Solutions = np.asarray(New_Solutions)
        All_Energies =[]
        for k in range(len(New_Solutions)): #Find for each chain strength
                #Energy function with 0 norm on lambda
                Energy = sparse_loss(Data_Patches[sample],Phi,np.asmatrix(New_Solutions[k]).T,lam)
                #print(Energy)
                All_Energies.append(Energy)
        index_min = np.argmin(np.asarray(All_Energies))
        Unembedded_Solutions_Training.append(New_Solutions[index_min])
        if image_number%10==0:
            print('Image Found', image_number)
        if image_number%100==0:
            print(New_Solutions[index_min])
        image_number+= 1
        
   


    Sparse_Rep = np.asarray(Unembedded_Solutions_Training)
    Sparse_Rep = np.asmatrix(np.squeeze(np.asarray(Sparse_Rep)))

    print('////////////////////////////////Sparse Reps Found\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\')
    Non_Zero_Coefficients.append(np.count_nonzero(Sparse_Rep))
    print('Non_Zero_Coefficient_Count',np.asarray(Non_Zero_Coefficients))
    #Unembedded_Solutions_Training = Sparse_Rep

    Temp_Data_Patches =np.asarray(Temp_Data_Patches)
    #Regular SGD
    Phi_New = Phi
    for i in range(mini_batch_size):
        error = np.subtract(Temp_Data_Patches[i].T,np.squeeze(np.dot(Phi,Unembedded_Solutions_Training[i].T)))
        Gradient = - (eta/mini_batch_size)*np.outer(error,Sparse_Rep[i]) + momentum_term*Previous_Gradient
        Phi_New = Phi_New + Gradient
        Previous_Gradient = Gradient

    #normalize Updated Phi

    for i in range(Neurons):
    #print((math.sqrt(Phi[:,i].T*Phi[:,i])))
        Phi_New[:,i]= Phi_New[:,i]/(math.sqrt(np.dot(Phi_New[:,i].T,Phi_New[:,i])))
    #print((math.sqrt(Phi[:,i].T*Phi[:,i])))
    if iteration ==0:
        Energies =[]
        for i in range(mini_batch_size):
            Energies.append(sparse_loss(Temp_Data_Patches[i],Phi,Sparse_Rep[i].T,lam))
        Energy = np.sum(np.asarray(Energies))/mini_batch_size
        Total_Energies.append(Energy)
    
    Energies =[]
    for i in range(mini_batch_size):
        Energies.append(sparse_loss(Temp_Data_Patches[i],Phi_New,Sparse_Rep[i].T,lam))
    Energy = np.sum(np.asarray(Energies))/mini_batch_size

    Total_Energies.append(Energy)

    count += 1

    #print(plt.plot(range(iteration+1), Total_Energies ))
    print('Energies,Epoc :', Total_Energies,count)
    elapsed1 = time.time() - t1
    print(elapsed1)

#     if Energy < Energy_Min:
        
#         Energy_Min = Energy
#         #renormalize phi
    sio.savemat('Phi_New_Fashion_MiniBatch1000_sPCA_32_Lambda_.1_RSGD_Undercomplete.mat', {'Phi_New':Phi_New})
    sio.savemat('TotalEnergies_Fashion_MiniBatch1000_sPCA_32_Lambda_.1_RSGD_Undercomplete.mat', {'Total_Energies':Total_Energies})
    if LA.norm(Phi_New-Phi) > tol:
        print(LA.norm(Phi_New-Phi))
        Phi = Phi_New


    else:
        print('converged')
        break






('Image Found', 0)
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
('Image Found', 10)
('Image Found', 20)
('Image Found', 30)
('Image Found', 40)
('Image Found', 50)
('Image Found', 60)
('Image Found', 70)
('Image Found', 80)
('Image Found', 90)
('Image Found', 100)
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
('Image Found', 110)
('Image Found', 120)
('Image Found', 130)
('Image Found', 140)
('Image Found', 150)
('Image Found', 160)
('Image Found', 170)
('Image Found', 180)
('Image Found', 190)
('Image Found', 200)
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
('Image Found', 210)
('Image Found', 220)


In [ ]:
T